In [ ]:
import pathlib
import random
import string
import re
import io

import numpy as np
import pandas as pd

from tensorflow import data as tf_data
from tensorflow import strings as tf_strings

import keras
from keras import ops
from keras import layers
from keras.layers import TextVectorization

In [15]:
path = pathlib.Path("merged_output.txt")
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

# max_size = 13_464_701
max_size = 1_000_000
# max_size = 500_000
# text = text[:max_size]

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 13464701
Total chars: 141
Number of sequences: 4488221


In [28]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        # layers.LSTM(128, return_sequences=True),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [30]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [43]:
epochs = 10
batch_size = 256

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % (epoch+1))

    start_index = random.randint(0, len(text) - maxlen - 1)
    if i % 5 == 0 or i == 1:
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print("...Diversity:", diversity)
    
            generated = ""
            sentence = text[start_index : start_index + maxlen]
            print('...Generating with seed: "' + sentence + '"')
    
            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.0
                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]
                sentence = sentence[1:] + next_chara
                generated += next_char
    
            print("...Generated: ", generated)
            print("-")

  238/17533 ━━━━━━━━━━━━━━━━━━━━ 27:32 96ms/step - loss: 2.1837

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
def generate_text(model, seed, maxlen, char_indices, indices_char, sample, diversity=1.0, generated_length=400):
    generated = ""
    sentence = seed
    print(f"...Generating with seed: \"{sentence}\"")

    for i in range(generated_length):
        x_pred = np.zeros((1, maxlen, len(char_indices)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        generated += next_char

    return generated


def run_text_generation(model, char_indices, indices_char, sample, maxlen, generated_length=400):
    print("Model is ready for text generation!")

    while True:
        seed = input("Enter seed text: ")

        if len(seed) > maxlen:
            print(f"Seed length should be less than or equal to {maxlen} characters.")
            continue

        # for diversity in [0.2, 0.5, 1.0, 1.2]:
        for diversity in [1.0]:
            print(f"...Diversity: {diversity}")
            generated_text = generate_text(model, seed, maxlen, char_indices, indices_char, sample, diversity)
            print(f"Generated text (diversity {diversity}):\n{generated_text}\n{'-' * 50}")

        cont = input("Do you want to generate another text (y/n)? ")
        if cont.lower() != 'y':
            break


In [ ]:
run_text_generation(model, char_indices, indices_char, sample, maxlen)